<a href="https://colab.research.google.com/github/BrithneyHernandez25/BrithneyHernandez.github.io/blob/main/CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
import pandas as pd
from google.auth import default
from datetime import datetime
creds, _ = default()
gc = gspread.authorize(creds)

# Define las URL de los archivos
urls = [
    "https://docs.google.com/spreadsheets/d/1h3GllUGEGkTLaFI1WdtmDLktXDKUysiMsdrQqmsQMW4/edit#gid=1207244430", # 2024
    "https://docs.google.com/spreadsheets/d/18r0uN5n2zducqllNN3Kl66PUPF_oLKFXLVtNYKwPlPw/edit#gid=214795300",# 2023
    "https://docs.google.com/spreadsheets/d/17jRq7Nq-X03mzRkIsLdKzW24FwYbyS-jyBVTrMifnf0/edit#gid=2003061115",#2022
    "https://docs.google.com/spreadsheets/d/1QS_4jeK0HuX67YYD3aVcYpgxttSQVWjJh5uYT1g6JuM/edit#gid=638747404" #2024 nuestro
    ]
# Lista para almacenar los DataFrames de cada hoja de cálculo
dfs = []

for url in urls:
    doc_id = url.split("/")[5]
    print("Documento ID:", doc_id)
    doc = gc.open_by_key(doc_id)
    ws = doc.worksheet("Coverage Tracker")
    print("Hoja de cálculo:", ws.title)  # Agregado para imprimir el título de la hoja
    # Obtener todos los valores de la hoja de cálculo
    values = ws.get_all_values()
    df = pd.DataFrame(values[2:], columns=values[1])  # Tomamos la segunda fila como encabezados
    # Eliminar filas vacías del DataFrame
    df.dropna(inplace=True)

    # Verificar si las columnas que queremos seleccionar existen en el DataFrame
    required_columns = [ 'URL / print', 'Outlet','Headline','Sentiment',  'Online / Print / TV / Radio', 'Primary Title','Tier', 'Date']
    for col in required_columns:
        if col not in df.columns:
            df[col] = ''  # Si la columna no existe, añadimos una columna vacía

    # Seleccionar las columnas necesarias y reordenar si es necesario
    df = df[required_columns]

    df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True) # Ordenar el DataFrame por la columna 'Date' de manera descendente
    df.sort_values(by='Date', ascending=False, inplace=True)

    # Agregar un espacio vacío si 'Outlet' o 'Headline' no están presentes en una fila determinada
    df['Outlet'].fillna('', inplace=True)
    df['Headline'].fillna('', inplace=True)
    df['Online / Print / TV / Radio'].fillna('', inplace=True)
    df['URL / print'].fillna('', inplace=True)
    df['Sentiment'].fillna('', inplace=True)
    df['Tier'].fillna('', inplace=True)
    df['Date'].fillna('', inplace=True)

    df = df[df['URL / print'] != '']  # Filtrar filas con URL / print vacías
     # Reemplazar "Paco Stanley" por "¿Quién lo mató?" en la columna 'Primary Title'
    df['Primary Title'] = df['Primary Title'].replace('Paco Stanley', '¿Quién lo mató?')
    df['Primary Title'] = df['Primary Title'].replace('Betty, la fea', 'Betty, la fea')
    df['Primary Title'] = df['Primary Title'].replace('Fallout', 'Fallout  S1')
    # df['Primary Title'] = df['Primary Title'].replace('Road House', 'Fallout  S1')

    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)
spreadsheet = gc.create('merged_data')# Crear una hoja de cálculo de Google
worksheet = spreadsheet.get_worksheet(0)  # Selecciona la primera hoja
merged_df['Date'] = merged_df['Date'].astype(str)# Convertir las columnas de fecha a cadenas de texto

# Actualizar la hoja de cálculo
worksheet.update([merged_df.columns.values.tolist()] + merged_df.values.tolist())
print("El DataFrame se ha guardado como una hoja de cálculo de Google.")

Documento ID: 1h3GllUGEGkTLaFI1WdtmDLktXDKUysiMsdrQqmsQMW4
Hoja de cálculo: Coverage Tracker


<ipython-input-48-24bd2f3c1534>:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True) # Ordenar el DataFrame por la columna 'Date' de manera descendente


Documento ID: 18r0uN5n2zducqllNN3Kl66PUPF_oLKFXLVtNYKwPlPw
Hoja de cálculo: Coverage Tracker
Documento ID: 17jRq7Nq-X03mzRkIsLdKzW24FwYbyS-jyBVTrMifnf0
Hoja de cálculo: Coverage Tracker
